# IBM Applied Data Science Capstone Course by Coursera
Week 5 Final Report

Opening a New Shopping Mall in Hyderabad, Telangana

* Build a dataframe of neighborhoods in Hyderabad, Telangana by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall

### 1. Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

! pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet


In [7]:
# print the number of rows of the dataframe
kl_df.shape

(200, 1)

### 3. Get the geographical coordinates

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, Telangana'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coordns = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [10]:
coordns

[[17.455470000000048, 78.54389000000003],
 [17.392977027745946, 78.45686724051741],
 [17.337590000000034, 78.56235000000004],
 [17.389800000000037, 78.47658000000007],
 [17.410600000000045, 78.51515000000006],
 [17.37751000000003, 78.48005000000006],
 [17.3873489754401, 78.4669932273483],
 [17.346330000000023, 78.47255000000007],
 [17.360690000000034, 78.47996000000006],
 [17.51908000000003, 78.41435000000007],
 [17.535430000000076, 78.54427000000004],
 [17.385800000000074, 78.51841000000007],
 [17.435340000000053, 78.44861000000003],
 [17.407860000000028, 78.49151000000006],
 [17.38513000000006, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.407430000000033, 78.50220000000007],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.399330000000077, 78.49965000000003],
 [17.341834499999997, 78.54754275],
 [17.448470000000043, 78.44937000000004],
 [17.41534000000007, 78.43437000000006],
 [17.386705398047635, 78.47824113930466],
 [17.3

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordns = pd.DataFrame(coordns, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coordns['Latitude']
kl_df['Longitude'] = df_coordns['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.455470,78.543890
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337590,78.562350
3,Abids,17.389800,78.476580
4,Adikmet,17.410600,78.515150
5,Afzal Gunj,17.377510,78.480050
6,Aghapura,17.387349,78.466993
7,"Aliabad, Hyderabad",17.346330,78.472550
8,Alijah Kotla,17.360690,78.479960
9,Allwyn Colony,17.519080,78.414350


In [16]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of Hyderabad with neighborhoods superimposed on top

In [18]:
# get the coordinates of Hyderabad
address = 'Hyderabad, Telangana'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, Telangana {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, Telangana 17.3616079, 78.4746286.


In [19]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [20]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [21]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ICQIK0WRHTQEBWFLI0TCRAW1GM1JQAXBASIRZUWSCR1KI1RS' # your Foursquare ID
CLIENT_SECRET = 'M2DYRCGBECXV20HL2ODWPOLIALQPFZJLELXLSVLTBDC3VSPZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Now, let's get the top 100 venues that are within a radius of 2000 meters. ####

In [22]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6778, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.45547,78.54389,Cinepolis CCPL,17.457282,78.536823,Multiplex
1,A. S. Rao Nagar,17.45547,78.54389,Raghavendra Cine Complex,17.451299,78.535697,Indie Movie Theater
2,A. S. Rao Nagar,17.45547,78.54389,Reliance Fresh,17.456698,78.545802,Supermarket
3,A. S. Rao Nagar,17.45547,78.54389,Green Bawarchi,17.457901,78.543900,Indian Restaurant
4,A. S. Rao Nagar,17.45547,78.54389,gossips bakery,17.456999,78.545902,Fast Food Restaurant


#### Let's check how many venues were returned for each neighorhood ####

In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,17,17,17,17,17,17
A.C. Guards,54,54,54,54,54,54
Abhyudaya Nagar,8,8,8,8,8,8
Abids,81,81,81,81,81,81
Adikmet,19,19,19,19,19,19
Afzal Gunj,46,46,46,46,46,46
Aghapura,53,53,53,53,53,53
"Aliabad, Hyderabad",16,16,16,16,16,16
Alijah Kotla,17,17,17,17,17,17


#### Let's find out how many unique categories can be curated from all the returned venues ####

In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 172 uniques categories.


In [28]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:150]

array(['Multiplex', 'Indie Movie Theater', 'Supermarket',
       'Indian Restaurant', 'Fast Food Restaurant', 'Tea Room', 'Café',
       'Neighborhood', 'Lake', 'Train Station', 'Bakery',
       'Hyderabadi Restaurant', 'Lounge', 'Juice Bar',
       'South Indian Restaurant', 'Bistro', 'Park', 'Science Museum',
       'Ice Cream Shop', 'Vegetarian / Vegan Restaurant', 'Hotel Bar',
       'Middle Eastern Restaurant', 'Pizza Place', 'Snack Place',
       'Stadium', 'Performing Arts Venue', 'Mobile Phone Shop', 'Hotel',
       'Coffee Shop', 'Fried Chicken Joint', 'Hookah Bar',
       'Department Store', 'Electronics Store', 'Clothing Store',
       'Farmers Market', 'Restaurant', 'Movie Theater',
       'Fruit & Vegetable Store', 'Diner', 'Food Truck', 'Shoe Store',
       'Chaat Place', 'Burger Joint', 'Dessert Shop', 'Smoke Shop',
       'Breakfast Spot', 'Food', 'Chinese Restaurant', 'Bar',
       'Shopping Mall', 'Gaming Cafe', 'Bus Station', 'Sandwich Place',
       'Food Court', 'C

In [29]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [30]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6778, 173)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pool,Pool Hall,Pub,Punjabi Restaurant,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Thai Restaurant,Theme Park,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category ####

In [32]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(200, 173)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Pool,Pool Hall,Pub,Punjabi Restaurant,Rajasthani Restaurant,Residential Building (Apartment / Condo),Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Thai Restaurant,Theme Park,Toy / Game Store,Trail,Train Station,Udupi Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.117647,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.176471,0.000000,0.117647,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.117647,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000
1,A.C. Guards,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.0

In [33]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

78

#### Create a new DataFrame for Shopping Mall data only ####

In [35]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [36]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,A. S. Rao Nagar,0.000000
1,A.C. Guards,0.000000
2,Abhyudaya Nagar,0.000000
3,Abids,0.012346
4,Adikmet,0.000000


### 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [37]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 2, 0], dtype=int32)

In [38]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [39]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,A. S. Rao Nagar,0.000000,0
1,A.C. Guards,0.000000,0
2,Abhyudaya Nagar,0.000000,0
3,Abids,0.012346,0
4,Adikmet,0.000000,0


In [40]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.455470,78.543890
1,A.C. Guards,0.000000,0,17.392977,78.456867
2,Abhyudaya Nagar,0.000000,0,17.337590,78.562350
3,Abids,0.012346,0,17.389800,78.476580
4,Adikmet,0.000000,0,17.410600,78.515150


In [41]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.455470,78.543890
119,Malkajgiri mandal,0.000000,0,17.447370,78.535200
120,Mallapur,0.000000,0,17.428700,78.551571
121,Mallepally,0.015152,0,17.391860,78.459050
123,Manovikas Nagar,0.000000,0,17.465610,78.493270
124,Marredpally,0.000000,0,17.447770,78.508730
125,Masab Tank,0.010000,0,17.400930,78.453630
127,Mehdipatnam,0.000000,0,17.392620,78.442210
128,Mettuguda,0.000000,0,17.434200,78.516360
118,Malkajgiri,0.000000,0,17.447370,78.535200


#### Finally, let's visualize the resulting clusters ####

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [43]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

Cluster 0

In [45]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.455470,78.543890
119,Malkajgiri mandal,0.000000,0,17.447370,78.535200
120,Mallapur,0.000000,0,17.428700,78.551571
121,Mallepally,0.015152,0,17.391860,78.459050
123,Manovikas Nagar,0.000000,0,17.465610,78.493270
124,Marredpally,0.000000,0,17.447770,78.508730
125,Masab Tank,0.010000,0,17.400930,78.453630
127,Mehdipatnam,0.000000,0,17.392620,78.442210
128,Mettuguda,0.000000,0,17.434200,78.516360
118,Malkajgiri,0.000000,0,17.447370,78.535200


Cluster 1

In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
22,Bairamalguda,0.200000,1,17.341834,78.547543
15,Attapur,0.125000,1,17.369170,78.436830
107,Lingojiguda,0.142857,1,17.349590,78.540330
189,Saroornagar,0.125000,1,17.346650,78.538650
131,Miyapur,0.142857,1,17.421020,78.582440
122,Manikonda,0.153846,1,17.401440,78.391660
60,Gaddiannaram,0.222222,1,17.351600,78.560710


Cluster 2

In [47]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
11,Amberpet,0.071429,2,17.385800,78.518410
52,ECIL 'X' Roads,0.058824,2,17.476590,78.568430
96,Kukatpally,0.100000,2,17.487350,78.420870
8,Alijah Kotla,0.058824,2,17.360690,78.479960
81,"Kamala Nagar, Hyderabad",0.043478,2,17.365600,78.533060
82,Kapra,0.057692,2,17.493570,78.401360
134,Moosapet,0.083333,2,17.466990,78.428420
30,Begum Bazaar,0.033333,2,17.373500,78.469050
5,Afzal Gunj,0.043478,2,17.377510,78.480050
93,"Kothapet, Hyderabad",0.076923,2,17.368820,78.542320


Most of the shopping malls are concentrated in the central area of Hyderabad city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.